<a href="https://colab.research.google.com/github/jsuj1th/Colab/blob/main/33500274_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### CSCE 670 :: Information Storage & Retrieval :: Texas A&M University :: Spring 2025


# Homework 1:  OASIS Search Engine, The Beginning

### 100 points [4% of your final grade]

### Due: February 5 (Wednesday) by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine + explore some LLM capabilities.

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via Canvas (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Canvas, search StackOverflow, even use ChatGPT. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. See the course syllabus for details.

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Dataset: Enron Email Dataset

We are providing you with a small collection of emails from the Enron Email Dataset. The Enron Email Dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes). It contains data from about 150 users, mostly senior management of Enron. The full corpus contains a total of about 0.5M messages (https://www.cs.cmu.edu/~enron/). For this homework, we will use a small subset of the data. The subset contains 814 emails extracted from the `_sent_mail` of Arnold-J. We have zipped the 814 files (each file contains the information of an email). The zipped file is available on Canvas as `enron_814.zip`. The subset we provide is about 1.1MB. You should treat each email as a unique document to be indexed by your system. You can download the data from Canvas to your local filesystem. We're going to use these emails as the basis of OASIS, our Open Access Searchable Information System!


Below is an example of one email.

```text
Message-ID: <33025919.1075857594206.JavaMail.evans@thyme>
Date: Wed, 13 Dec 2000 13:09:00 -0800 (PST)
From: john.arnold@enron.com
To: slafontaine@globalp.com
Subject: re:spreads
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: John Arnold
X-To: slafontaine@globalp.com @ ENRON
X-cc:
X-bcc:
X-Folder: \John_Arnold_Dec2000\Notes Folders\'sent mail
X-Origin: Arnold-J
X-FileName: Jarnold.nsf

saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.
```

# Part 1: Read and Parse the Email Data (20 points)

Recall how we handled file input in Homework 0? Well, here, our goal is to read the emails so that we can begin to tokenize them later. For this step, you should read the dataset and print the emails. Note that our dataset contains multiple files. You will need to write Python code to read from these files, and then build a list to store the documents. Each item in the list should be a dictionary containing the `Document-ID` as the key, and email content as the value. You can discard all the supplementary information of the email, e.g., `Date`, `From`, `To`, `Subject`, etc.

A document should look like:

```text
{'Document-ID': '33025919.1075857594206',
'content': 'saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.'
}
```

For this homework, you should treat the email content as a document and the Message-ID as the document ID.

## Print the first two documents (5 points)

Your output should look like this:

DocumentID Document

33025919.1075857594206 saw a lot of the bulls sell summer ......

...


In [3]:
!apt-get install p7zip-full
!7z x enron_814.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1036170 bytes (1012 KiB)

Extracting archive: enron_814.zip
--
Path = enron_814.zip
Type = zip
Physical Size = 1036170

  0%     54% 933 - __MACOSX/enron_814/._688.                                     89% 1468 - enron_814/548.                          Everything is Ok

Folders: 1
Files: 1629
Size:       1334204
Compressed: 1036170


In [4]:
# your code here
# please print out the first 2 docs
import re, os, json
from itertools import islice
Mails=[]
doc={}
i=1
for file in os.listdir("enron_814"):
  # print(file)

  with open(os.path.join("enron_814",file), 'r', encoding='utf-8') as f:
    mail= f.read()

    match = re.search(r"(\d+\.\d+)", mail)
    content_match=re.search(r"(\r?\n){2,}([\s\S]+)", mail)
    if match:
      document_id = match.group(1)
      content=content_match.group(2)
      doc[document_id]=content



Now that you can read the documents, let's move on to tokenization. We are going to simplify things for you:
1. You should **lowercase** all words.
2. Replace line breaks (e.g., \n, \n\n), punctuations, dashes and splash (e.g., -, /) and special characters (\u2019, \u2005, etc.) with empty space (" ").
3. Tokenize the documents by splitting on whitespace.
4. Then only keep words that have a-zA-Z in them.

In [5]:
# your code here

def custom_tokenize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'[\n\r\t.:]', ' ', text)
    text=re.sub(r'[^a-zA-Z\s]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    return tokens

tokens=[]
for id,mail in doc.items():
  tks=custom_tokenize(mail)
  for tk in tks:
    tokens.append(tk)
print(len(set(tokens)))
lookup_doc=doc.copy()
# print(doc)
# print(lookup_doc)


9410


## Print the first two documents after tokenizing (5 points)

Once you have your parser working, you should print the first two documents (documentID and tokens).

Your output should look like this:

* DocumentID Tokens



In [6]:
# your code and output here
for id,mail in doc.items():
  doc[id]=custom_tokenize(mail)
# print(lookup_doc)

In [7]:

for key, value in islice(doc.items(), 2):
    print(key, value)

20191135.1075857601260 ['hey', 'can', 'greg', 'fit', 'me', 'in', 'for', 'about', 'minutes', 'tomorrow', 'afternoon', 'your', 'secret', 'admirer']
26241159.1075857650422 ['how', 'about', 'we', 'celebrate', 'the', 'near', 'completion', 'of', 'your', 'deal', 'and', 'have', 'tickleless', 'pay', 'for', 'it', 'from', 'kim', 'ward', 'enron', 'enronxgate', 'on', 'pm', 'to', 'john', 'arnold', 'hou', 'ect', 'ect', 'cc', 'subject', 're', 'push', 'the', 'summer', 'down', 'about', 'so', 'i', 'can', 'get', 'one', 'of', 'my', 'deals', 'done', 'and', 'we', 'could', 'celebrate', 'original', 'message', 'from', 'arnold', 'john', 'sent', 'tuesday', 'may', 'pm', 'to', 'ward', 'kim', 's', 'subject', 'wanna', 'get', 'sauced', 'after', 'work']


## Dictionary Size (5 points)

Next you should report the size of your dictionary, that is, how many unique tokens among all the documents.




In [8]:
print("No.of Unique Tokens:",len(set(tokens)))
len(tokens)


No.of Unique Tokens: 9410


127149

## Top-20 Words (5 points)

Finally, you should print a list of the top-20 most popular words by counting among all documents.


Your output should look like this:

* Rank. Token, Count

1. awesome, 20
2. cool, 15
3. ...

In [9]:
from collections import Counter
word_counts = Counter(tokens)
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
sorted_word_counts[:20]


[('the', 4111),
 ('to', 3768),
 ('ect', 2600),
 ('enron', 2296),
 ('a', 1897),
 ('and', 1795),
 ('you', 1749),
 ('of', 1716),
 ('i', 1696),
 ('on', 1590),
 ('john', 1514),
 ('in', 1465),
 ('hou', 1345),
 ('com', 1319),
 ('is', 1202),
 ('for', 1165),
 ('arnold', 999),
 ('subject', 936),
 ('s', 871),
 ('it', 865)]

# Part 2: Boolean Retrieval (30 points)

In this part you will build an inverted index to support Boolean retrieval. You should use the tokenization strategy from above.

We require your index to support AND, OR, NOT queries.

Search for the queries below using your index and print out matching documents (for each query, print out 5 matching documents):
* buyer
* margins AND limits
* winter OR summer
* buyers AND risk AND NOT crazy
* never OR know

Recall, that you should apply the exact same pre-processing strategies to the query as we do to the documents.

The output should like this:
* DocumentID Document

To make our life easier, please output the DocumentIDs in lexicographic order.

In [10]:
# build the index here
# add cells as needed to organize your code

##Building inverted Index
inv_index={}
for key, tokens in doc.items():
  for token in tokens:
    if token not in inv_index:
      inv_index[token]=[]
    inv_index[token].append(key)





In [11]:
# def retrieve_docs_only(query):
#   query=query.split()
#   print(query)
#   retrieved_docs=[]
#   for i in range(len(query)):
#     # print(retrieved_docs)
#     if i%2==0:
#       if query[i] not in inv_index:
#         return []
#       else:
#         retrieved_docs.append(inv_index[query[i]])
#     # elif i%2==1:
#     if query[i]=='NOT':
#       retrieved_docs.append(list(set(doc.keys())-set(inv_index[query[i]])))
#       del query[i]
#       print(query)
#   return retrieved_docs
# def perform_operations(query, retrieved_docs):
#   if type(query)== str:
#     query=query.split()
#   if len(query)<=1:
#     return retrieved_docs
#   # print(query)
#   if query[1]== 'OR':
#     retrieved_docs[0]=set(retrieved_docs[0]+retrieved_docs[1])
#   elif query[1]== 'AND':
#     # print(retrieved_docs)
#     retrieved_docs[0]=list(set(retrieved_docs[0]).intersection(retrieved_docs[1]))
#   # elif query[1]== 'NOT':
#   #   retrieved_docs[0]=list(set(retrieved_docs[0])-set(retrieved_docs[1]))
#   # return retrieved_docs[0]
#   if len(retrieved_docs) >= 2:del retrieved_docs[1]
#   del query[:3]
#   # print(retrieved_docs)
#   return perform_operations(query, retrieved_docs)
# query="buyers AND risk AND NOT crazy"
# retrieved_docs=retrieve_docs_only(query)
# # print(retrieved_docs)
# r_doc_id = list(perform_operations(query, retrieved_docs)[0])
# print(r_doc_id)
# for doc_id in r_doc_id:
#   print(doc_id, lookup_doc[doc_id])




In [12]:
# print(lookup_doc)

## Running the five queries (4 points each, 20 points in total)

In [13]:
def preprocess_query(query):
    return query.lower().split()

def and_query(inv_index, terms):
    if not terms:
        return []
    processed_result = set(inv_index.get(terms[0], []))
    for term in terms[1:]:
        processed_result &= set(inv_index.get(term, []))
    return list(processed_result)

def or_query(inv_index, terms):
    processed_result = set()
    for term in terms:
        processed_result |= set(inv_index.get(term, []))
    return list(processed_result)

def not_query(inv_index, term, all_docs):
    docs_with_term = set(inv_index.get(term, []))
    return list(all_docs - docs_with_term)

def parse_query(inv_index, query, all_docs):
    terms = preprocess_query(query)
    if not terms:
        return list(all_docs)
    i = 0
    processed_terms = []
    while i < len(terms):
        if terms[i] == "not" and i + 1 < len(terms):
            not_processed_result = set(not_query(inv_index, terms[i + 1], all_docs))
            processed_terms.append(("TERM", not_processed_result))
            i += 2
        else:
            if terms[i] not in ("and", "or"):
                term_docs = set(inv_index.get(terms[i], []))
                processed_terms.append(("TERM", term_docs))
            else:
                processed_terms.append(("OP", terms[i]))
            i += 1
    i = 0
    and_processed = []
    while i < len(processed_terms):
        if i == 0 and processed_terms[i][0] != "OP":
            and_processed.append(("TERM", processed_terms[i][1]))
            i += 1
        elif i+1 < len(terms) and processed_terms[i][1] == "and":
            left_type, left_docs   = and_processed[-1]
            right_type, right_docs = processed_terms[i+1]
            if left_type in ("TERM") and right_type in ("TERM"):
                and_processed_result        = left_docs & right_docs
                and_processed[-1] = ("TERM", and_processed_result)
                i += 2
            else:
                and_processed.append(processed_terms[i])
                i += 1
        else:
            and_processed.append(processed_terms[i])
            i += 1
    i = 0
    final_processed = []
    while i < len(and_processed):
        if i == 0 and and_processed[i][0] != "OP":
            final_processed.append(("TERM", and_processed[i][1]))
            i += 1
        elif i+1 < len(terms) and and_processed[i][1] == "or":
            left_type, left_docs   = final_processed[-1]
            right_type, right_docs = and_processed[i+1]
            if left_type in ("TERM") and right_type in ("TERM"):
                or_processed_result           = left_docs | right_docs
                final_processed[-1] = ("TERM", or_processed_result)
                i += 2
            else:
                final_processed.append(and_processed[i])
                i += 1
        else:
            final_processed.append(and_processed[i])
            i += 1
    return list(set(final_processed[-1][1]))

Now show the results for the query: `buyer`

In [14]:
# your code here
query="buyer"
parse_query(inv_index, query, set(doc.keys()))

['28376645.1075857655238',
 '25732708.1075857656969',
 '5307647.1075857657213',
 '17195279.1075857655281',
 '3677051.1075857657147',
 '33025919.1075857594206',
 '13960264.1075857658698',
 '2268604.1075857652949',
 '5304428.1075857597649']

*Now* show the results for the query: `margins AND limits`

---




In [15]:
# your code here
query="margins AND limits"
parse_query(inv_index, query, set(doc.keys()))

['33025919.1075857594206']

Now show the results for the query: `winter OR summer`

In [16]:
# your code here
query="winter OR summer"
parse_query(inv_index, query, set(doc.keys()))

['31307130.1075857655193',
 '26583745.1075857594731',
 '2363438.1075857657841',
 '12760800.1075857651684',
 '26173217.1075857594537',
 '33491127.1075857594966',
 '5307647.1075857657213',
 '1475999.1075857651728',
 '33358932.1075857653571',
 '13960264.1075857658698',
 '18520811.1075857650486',
 '17805507.1075857658940',
 '18731712.1075857658964',
 '11971129.1075857654956',
 '2268604.1075857652949',
 '19675930.1075857656429',
 '30030513.1075857650336',
 '3126127.1075857653305',
 '14369391.1075857597800',
 '13876367.1075857655500',
 '23667211.1075857656689',
 '24632457.1075857653283',
 '29406139.1075857594335',
 '6398802.1075857654979',
 '15042778.1075857600714',
 '10353423.1075857652669',
 '33025919.1075857594206',
 '19850187.1075857654782',
 '14371860.1075857601017',
 '10603457.1075857597605',
 '32415330.1075857655843',
 '773676.1075857654425',
 '19235579.1075857594400',
 '17721019.1075857596305',
 '23931433.1075857654804',
 '30621564.1075857650358',
 '27427045.1075857651911',
 '1803263

Now show the results for the query: `buyers AND risk AND NOT crazy`

In [17]:
# your code here
query="buyers AND risk AND NOT crazy"
parse_query(inv_index, query, set(doc.keys()))

['2726985.1075857655016']

Now show the results for the query: `never OR know`

In [18]:
# your code here
query="never OR know"
parse_query(inv_index, query, set(doc.keys()))

['31307130.1075857655193',
 '26813632.1075857653239',
 '6099369.1075857598441',
 '6384662.1075857656041',
 '8618376.1075857651868',
 '22083600.1075857596413',
 '19835539.1075857596349',
 '18520811.1075857650486',
 '2268604.1075857652949',
 '18731712.1075857658964',
 '7293239.1075857597953',
 '3574764.1075857600475',
 '3126127.1075857653305',
 '23057737.1075857601038',
 '13876367.1075857655500',
 '17259122.1075857598177',
 '3073341.1075857596866',
 '18917089.1075857653085',
 '20679245.1075857597281',
 '10353423.1075857652669',
 '27187948.1075857650684',
 '26270100.1075857598045',
 '10396784.1075857651750',
 '32776644.1075857658676',
 '773676.1075857654425',
 '25124831.1075857599419',
 '2200629.1075857599483',
 '7410631.1075857598982',
 '10008095.1075857595829',
 '28376645.1075857655238',
 '18346325.1075857654266',
 '17569138.1075857658721',
 '17195279.1075857655281',
 '23707366.1075857595937',
 '13443589.1075857600218',
 '1837490.1075857597060',
 '21351833.1075857600305',
 '1654017.1075

## Observations (10 points)
Does your boolean search engine find relevant documents for these queries? As in, would our customers be happy if we shipped this retrieval engine? Why or why not?

What is the impact of the pre-processing options? Do they impact your search quality?

*your discussion here*

#### The search engine is fetching relevant documents for the queries. The customers will be happy as it makes their search easy and the search engine has the capability for long queries. So, the user can modify their search for exact results.

# Part 3: Ranking (35 points)

For the third part, you will add ranking to your search system. Given a search query, our goal is to retrieve the top-5 most relevant emails by assigning a score to each document.

We will explore two ranking methods, each offering a different approach to scoring and ranking documents:


### Ranking method A: Ranking with vector space model with TF-IDF (15 points)

**Cosine:** You should use cosine as your scoring function.

**TFIDF:** For the **document vectors**, use the standard TF-IDF scores introduced in class. For the **query vector**, use **simple weights (the raw term frequency)**. For example:
* query: never $\rightarrow$ (1)
* query: never know $\rightarrow$ (1, 1)

**Query:**  `trade`

**Output:**
You should output the top-5 results plus the cosine score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

---

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!


In [44]:
# your code here
# print(word_counts)
from collections import defaultdict
import math
# for key, value in islice(doc.items(), 2):
#     print(key, value)

def tfidf(doc):
  doc_feq=defaultdict(int)
  tfidf_scores={}
  for tks in doc.values():
    for token in set(tks):
      doc_feq[token]+=1
  for doc_id, tks in doc.items():
      tfidf_vector={}
      total_tks=len(tks)
      for t in tks:
        tf=tks.count(t)/total_tks
        idf=math.log(len(doc)/doc_feq[t])
        tfidf_vector[t]=tf*idf
      # doc[doc_id]=tfidf_vector
      tfidf_scores[doc_id]=tfidf_vector
  return tfidf_scores

def cosine_similarity(query_vector, doc_vector):
    common_keys=set(query_vector.keys()).intersection(set(doc_vector.keys()))
    # for key in common_keys:
    dot_product=sum(query_vector[key]*doc_vector[key] for key in common_keys)
    query_magnitude=math.sqrt(sum(query_vector[key]**2 for key in query_vector))
    doc_magnitude=math.sqrt(sum(doc_vector[key]**2 for key in doc_vector))
    if query_magnitude==0 or doc_magnitude==0:
      return 0
    return dot_product/(query_magnitude*doc_magnitude)

tfidf_scores=tfidf(doc)
# print(tfidf_scores)
query="trade"
query_vector={}
query_token=custom_tokenize(query)
for key in query_token:
  query_vector[key]=1
print(query_vector)
scores={}
for doc_id, doc_vector in tfidf_scores.items():
  scores[doc_id]=cosine_similarity(query_vector, doc_vector)
ranked_results=sorted(scores.items(), key=lambda x: (x[1],x[0]), reverse=True)[:5]
# print(ranked_results)
for i in range(len(ranked_results)):
  print("Rank:",i+1, "cosine_score:",ranked_results[i][1], "document_id:",ranked_results[i][0], "\n", lookup_doc[ranked_results[i][0]])
  print("\n-------------------------------------------------")







# print out the top-5 retrieved emails


{'trade': 1}
Rank: 1 cosine_score: 0.35268279961678006 document_id: 32835197.1075857597302 
 Hey:
I just want to confirm the trades I have in your book.
Trade #1.  I sell 4000 X @ 4652

Trade #2. I buy 4000 X @ 4652
  I sell 4000 X @ 4902

Trade #3 I buy 4000 X @ 5000
  I sell 4000 F @ 5000


Net result: I have 4000 F in your book @ 4902.
Thanks, 
John

-------------------------------------------------
Rank: 2 cosine_score: 0.3053426239418323 document_id: 15827855.1075857658654 
 torrey:
please set me up to trade crude.
John

-------------------------------------------------
Rank: 3 cosine_score: 0.2283779396714838 document_id: 2752057.1075857658632 
 Torrey:
Can you also approve Mike Maggi to trade crude as well.  Thanks for your help.
John

-------------------------------------------------
Rank: 4 cosine_score: 0.22160785591147464 document_id: 3383202.1075857656796 
 you fucker that's my trade.   i was trying to buy nines the last 20 minutes.  
all i got was scraps.  50-100.  i think

### Ranking method B: Ranking with BM25 (15 points)
Finally, let's try the BM25 approach for ranking. Refer to https://en.wikipedia.org/wiki/Okapi_BM25 for the specific formula. You could choose k_1 = 1.2 and b = 0.75 but feel free to try other options.

**Query:**  `gas floor`

**Output:**
You should output the top-5 results plus the BM25 score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

In [20]:
# your code here


# print out the top-5 retrieved emails

## Observations (5 points)
What do you observe? Are there key differences between the two ranking approaches? Briefly discuss in bullet points.


* Your observations:
* Differences:

# Part 4: Cool LLM RAG Extension (15 points)

Finally, we give you an opportunity to explore using OASIS for Retrieval-Augmented Generation (RAG) with LLMs.
Here, the task is to retrieve the top-5 emails for a query you like. You will then pass the retrieved email content along with your question to the LLM and let it answer your question. Specifically, we want you:

* Query the LLM directly with your question.
* Retrieve the top-5 emails based on your query and pass them to the LLM along with your question.
* How is the RAG output different from the non-RAG output? Does retrieval help the LLM better answer your question?

We recommend using Gemini following the [instructions](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb) here.

Hint: Take a close look at the dataset and pick a specific, relevant query where retrieval can enhance the LLM’s response.

*What You Will Submit*
- Your query.
- The top-5 retrieved emails (including Document ID and ranking score).
- The LLM's response without retrieval.
- The LLM's response with retrieval (RAG).
- A brief analysis comparing both outputs.

We will grade this last part according to correctness, effort, and creativity.

## Step 1: Query the LLM Directly (Without Retrieval) (5 points)

In [21]:
# your code here to prompt LLM

## Step 2: Query the LLM with Retrieved Emails (RAG) (5 points)

In [22]:
# print out the LLM response without the email content


# print out the LLM response with the email content (RAG)



## Discussion (5 points)

In this section, reflect on the performance of different ranking methods and the impact of retrieval on LLM responses. Consider:

- How did retrieval affect the LLM’s response? Did it improve factual accuracy or relevance?
- Were there cases where retrieval hurt performance (e.g., irrelevant documents, redundancy)?
- Any ideas for improving the ranking or retrieval process?

Keep your discussion *concise* and *insightful*, focusing on key takeaways from your experiments. Please answer in bullet points.

*your discussion here*

* point 1
* point 2
* ...

# Collaboration Declarations

*You should fill out your collaboration declarations here.*

https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python


https://www.geeksforgeeks.org/python-difference-two-lists/

ChatGPT:
- i want to no. of occurences of each element in a list